In [22]:
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)

from llama_index.legacy.llms.vertex import Vertex
from langchain.chat_models import ChatVertexAI
from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy import Document
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser


from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import chromadb


from google.oauth2 import service_account
from llama_index.legacy import set_global_service_context
from google.cloud import aiplatform
import json 

## imports from og llama index file

In [23]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy.text_splitter import TokenTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.llms.vertex import Vertex
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)
from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

from llama_index.legacy.postprocessor import NERPIINodePostprocessor, SentenceEmbeddingOptimizer
from llama_index.legacy import ServiceContext
from llama_index.legacy.schema import QueryBundle
from llama_index.legacy.schema import NodeWithScore, TextNode

from pathlib import Path
from tqdm.notebook import tqdm
from google.oauth2 import service_account

from llama_index.legacy import set_global_service_context
import re
import uuid
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
import csv
from typing import List, Tuple, Dict
import time
import json

from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.legacy import StorageContext
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import ChatVertexAI
from llama_index.legacy import Response
from llama_index.legacy.response_synthesizers import Refine
from llama_index.legacy.evaluation import SemanticSimilarityEvaluator
from llama_index.legacy.evaluation import RelevancyEvaluator
from llama_index.legacy.embeddings import SimilarityMode

import nest_asyncio
nest_asyncio.apply()

# additional
from llama_index.legacy.postprocessor import LongContextReorder
from llama_index.legacy.retrievers import AutoMergingRetriever
from llama_index.legacy.query_engine import TransformQueryEngine
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.indices.query.query_transform.base import (
    HyDEQueryTransform,StepDecomposeQueryTransform
)
from llama_index.legacy.node_parser import get_leaf_nodes
from llama_index.legacy import Document
# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
# callback_manager = CallbackManager([llama_debug])

## Initialize credentials, models, and embedding models

In [24]:
import vertexai
from vertexai.language_models import TextGenerationModel
from langchain_google_vertexai.llms import VertexAI

credentials = service_account.Credentials.from_service_account_file("credentials/vertex-test-417403-ce72ad032af7.json")
vertex_ai = Vertex(model="text-bison", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
text_model = TextGenerationModel.from_pretrained("text-bison@002")
text_vertex_ai = VertexAI(model_name="text-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)                       
chat_vertex_ai = ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
langchain_chat_vertex_ai = LangChainLLM(ChatVertexAI(model_name="chat-bison@002", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2))

selected_model = langchain_chat_vertex_ai

embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest'))

In [25]:
service_context = ServiceContext.from_defaults(llm=vertex_ai, embed_model=embed_model, chunk_size=1024, chunk_overlap=20)
set_global_service_context(service_context)
# need to set global service context becuase if not index = VectorStoreIndex.from_vector_store(vector_store, embed_model=Settings.embed_model) will produce an error
# it will say that it requires OPENAI key etc
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='textembedding-gecko-multilingual@latest', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000002D8EB24EA50>, id_func=<function default_id_func at 0x000002D8D4E38860>, chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x000002D8EEA36D90>, callback_manager=<llama_index.legacy.callbacks.base.

In [26]:
import log_appointment as logapp # pingpings file

## {DONE} Produce csv storing query, model response (and soon others)

load query text files into one csv file

In [27]:
# import pandas as pd

# # Sample text strings (replace with your actual file paths)
# text_file1 = "../data/chief complaint.txt"
# text_file2 = "../data/making appointment.txt"
# text_file3 = "../data/medical experts.txt"

# # Create a list of text strings
# text_list = []
# for filename in [text_file1, text_file2, text_file3]:
#   with open(filename, "r") as f:
#     # Read all lines and concatenate them into a single string
#     text_content = f.readlines()  # Remove potential leading/trailing whitespace
#     text_list.extend(text_content)

# text_list = [line.replace('\n', '') for line in text_list]
# print(len(text_list))
# text_list

In [28]:
# # Create a DataFrame with a single column named "Text"
# # df = pd.DataFrame({"Text": text_list}, columns=["query"])
# df = pd.DataFrame(text_list, columns=["query"])

# filepath = "model_response.csv"
# df.to_csv(filepath, index=False)
# print(f"Data saved to {filepath}")

Create dataframe from model_response.csv file

In [29]:
# filepath = "model_response.csv"
# model_response_df = pd.read_csv(filepath)

Create qna dataframe from medical_experts_ans.csv (for appointments)

In [52]:
filepath = "medical_experts_ans.csv"
model_response_df = pd.read_csv(filepath)
model_response_df

,query,ref_answer
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...


Create log_appointment.py log file

In [31]:
log_path = "../data/log_appointment.csv"
logapp.create_appoint_log(num_week=5, log_path=log_path, delete_existing=True)

## Import and load data
- How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

### Load data from doctor_db.csv (modified from siriraj_doctor_details.csv for appointments) 

In [32]:
import pandas as pd

# unused
# reader = SimpleDirectoryReader(
#     input_files=["../data/siriraj_doctor_details.csv"]
# )
# csv_data = reader.load_data()
# print(f"Loaded {len(csv_data)} docs")
# csv_data

csv_file_path = "../data/doctor_db.csv"
csv_df = pd.read_csv(csv_file_path)
columns_to_drop = ['image_src', 'url', 'qualification']
csv_df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
csv_df

,name,table_check,expertise,doctorID
0,รศ. พญ.กติกา นวพันธุ์,"เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",เวชศาสตร์มารดาและทารกในครรภ์,0
1,รศ. พญ.กนกรัตน์ พรพาณิชย์,"พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5",ศัลยกรรมกระดูกเบ้าตาและตกแต่ง,1
2,อ. พญ.กนกรัตน์ สุวรรณสิทธิ์,"พฤหัสบดี 13:30 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",โสต ประสาท การได้ยินและการทรงตัว,2
3,ทพญ.กนกอร ตันติพาณิชย์กูล,"พุธ 16:30 - 19:30 น. สัปดาห์ที่ 3,4",ทันตกรรมจัดฟัน การจัดฟันด้วยการผ่าตัดจัดฟัน,3
4,ผศ. นพ.กมล เผือกเพ็ชร,"พุธ 16:00 - 18:00 น. สัปดาห์ที่ 1,3,5 อาทิตย์ ...",กุมารเวชศาสตร์โลหิตวิทยาและมะเร็งในเด็ก,4
...,...,...,...,...
625,นพ.เอกราช ลิ่วธนมงคล,"เสาร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 13...","โรคของลำไส้ใหญ่และทวารหนัก, ฝีคัณฑสูตร, ริดสีด...",625
626,รศ. นพ.เอกรินทร์ โชติกวาณิชย์,"พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร...","โรคนิ่วไต ท่อไต และกระเพาะปัสสาวะ, โรคเนื้องอก...",626
627,อ. นพ.เอกวิทย์ เกยุราพันธุ์,"ศุกร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 อา...",ศัลยศาสตร์ออร์โธปิดิกส์,627
628,นพ.เอนกชัย ดกพฤกษ์,NaN,เวชศาสตร์ฉุกเฉิน,628


### {UNUSED} Load data from siriraj_doctor_details.csv (given)

Cleaning csv data

In [33]:
# import pandas as pd

# # unused
# # reader = SimpleDirectoryReader(
# #     input_files=["../data/siriraj_doctor_details.csv"]
# # )
# # csv_data = reader.load_data()
# # print(f"Loaded {len(csv_data)} docs")
# # csv_data

# csv_file_path = "../data/siriraj_doctor_details.csv"
# csv_df = pd.read_csv(csv_file_path)
# columns_to_drop = ['image_src', 'url', 'qualification']
# df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
# df

### TODO: Cluster expertise (not done) << FULL CODE IS IN MINNIELLM. IT IS REMOVED IN THIS FILE
- not sure if needed atm. will be used as a point of improvement later

In [34]:
csv_data = csv_df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
csv_data = [Document(text=t) for t in csv_data] # produce Documents from csv data
print(len(csv_data))
print(csv_data)

630
[Document(id_='b71286fa-9cd2-46dd-8f52-d139de0895f3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กติกา นวพันธุ์เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5เวชศาสตร์มารดาและทารกในครรภ์0', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='a3429ea8-16e3-4d4f-bc93-7baa21e82634', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กนกรัตน์ พรพาณิชย์พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5ศัลยกรรมกระดูกเบ้าตาและตกแต่ง1', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='cd64b9ee-b258-46bd-9f6b-d562c2047730', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, te

### Combine all loaded data into singular Document list docs

In [35]:
docs = []
docs.extend(csv_data)

print(len(docs))
print(docs)

630
[Document(id_='b71286fa-9cd2-46dd-8f52-d139de0895f3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กติกา นวพันธุ์เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5เวชศาสตร์มารดาและทารกในครรภ์0', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='a3429ea8-16e3-4d4f-bc93-7baa21e82634', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กนกรัตน์ พรพาณิชย์พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5ศัลยกรรมกระดูกเบ้าตาและตกแต่ง1', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='cd64b9ee-b258-46bd-9f6b-d562c2047730', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, te

### Create local ChromaDB Vectore Store (currently unused)

In [36]:
# # Create index and persist to Chroma vector store

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents = docs, storage_context=storage_context, embed_model=embed_model
# )

In [37]:
# Load index from persisted vector store

# currently index is used for retriever and not query engine

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# chromadb_index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

# chromadb_query_engine = chromadb_index.as_query_engine(similarity_top_k=2)
# vector_retriever = chromadb_index.as_retriever(similarity_top_k=2)

### Configuring query engine components

Hierarchical Node Parsing

In [38]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[1024, 512, 256]
)
nodes = node_parser.get_nodes_from_documents(docs)

leaf_nodes = get_leaf_nodes(nodes)
nodes_by_id = {node.node_id: node for node in nodes}

auto_merging_context = ServiceContext.from_defaults(
    llm=selected_model,
    embed_model= embed_model,
    node_parser=node_parser,
)

Store configured components in vector store called merging_index

In [39]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)
siriraj_doctor_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)

# TODO: fix the code to store in chromadb
# currently chromadb is not used

# merging_index_dir = "./merging_index"
siriraj_doctor_index_dir = "./siriraj_doctor_index"
siriraj_doctor_index.storage_context.persist(persist_dir=siriraj_doctor_index_dir)

Load automerging index

In [40]:
# merging_index_dir = "./merging_index"
siriraj_doctor_index_dir = "./siriraj_doctor_index"
# loaded_storage_context = StorageContext.from_defaults(persist_dir=merging_index_dir) # load the existing index
loaded_storage_context = StorageContext.from_defaults(persist_dir=siriraj_doctor_index_dir) # load the existing index
# automerging_index = load_index_from_storage(loaded_storage_context)
siriraj_doctor_index = load_index_from_storage(loaded_storage_context)


In [41]:
# automerging_as_retriever = automerging_index.as_retriever(similarity_top_k=10)
# automerging_retriever = AutoMergingRetriever(
#     automerging_as_retriever, 
#     automerging_index.storage_context, 
#     verbose=True
# )

automerging_as_retriever = siriraj_doctor_index.as_retriever(similarity_top_k=10)
automerging_retriever = AutoMergingRetriever(
    automerging_as_retriever, 
    siriraj_doctor_index.storage_context, 
    verbose=True
)

### Query engine

In [42]:
reorder = LongContextReorder()
hyde = HyDEQueryTransform(llm=vertex_ai, include_original=True)
retriever_query_engine = RetrieverQueryEngine.from_args(automerging_retriever,
                                              node_postprocessors=[reorder],
                                              )
transform_query_engine = TransformQueryEngine(retriever_query_engine, query_transform=hyde)

TESTING: Response production

In [43]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response=transform_query_engine.query(query)
print("response: ", response.response)
# print("source nodes: ", response.source_nodes[0])
# print("source node text: ", response.source_nodes[0].node.text)
# print("number of source nodes: ", len(response.source_node))

> Merging 1 nodes into parent node.
> Parent node id: 0669f3f3-2e62-44e7-b18b-7c0fcaa2cc49.
> Parent node text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43

> Merging 1 nodes into parent node.
> Parent node id: 4ee8b116-f5c1-4d00-8766-4ca303124b09.
> Parent node text: พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดินหายใจ535

> Merging 1 nodes into parent node.
> Parent node id: 41778c05-2960-48fd-990e-1dba25995d4b.
> Parent node text: ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ โรคปอดและเวชศาสตร์วิกฤต352

> Merging 1 nodes into parent node.
> Parent node id: 6b65ec66-0337-48f0-864d-118480114e14.
> Parent node text: ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:00 - 11:00 น...

> Merging 1 nodes into parent node.
> Parent node id: 34a8ea8f-bc95-48ba-935e-f977fda6538c.
> Parent node text: รศ. นพ.ชุณหเกษม   โชตินัยวัตรกุลอังคาร 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 13:00 - 16:00

Inspecting response and context nodes

In [44]:
response.response

' ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์'

In [45]:
print(response.source_nodes[0])

Node ID: 6d5ae2d2-7a4d-49e8-8ec2-5034d2ecc7cc
Text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด,
หลอดเลือดหัวใจตีบและลิ้นหัวใจ43
Score:  0.725



In [46]:
response.source_nodes[0].node.text

'รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43'

In [47]:
len(response.source_nodes)

10

Context Transformations - PII Example
- adding context to model response generation

In [48]:
pii_processor = NERPIINodePostprocessor()

def filter_pii_fn(**kwargs):
    # run optimizer
    query_bundle = QueryBundle(query_str=kwargs["query_str"])

    new_nodes = pii_processor.postprocess_nodes(
        [NodeWithScore(node=TextNode(text=kwargs["context_str"]))],
        query_bundle=query_bundle,
    )
    new_node = new_nodes[0]
    return new_node.get_content()

In [49]:
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "(day) (time) (full name of doctor) (doctor's expertise)\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    # "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl_str2 = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If the query has date time, "
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment, answer in the following format only.\n"
    "{day_time} {full_name} ({expertise})\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
    "1. {day_time} {full_name} ({expertise})\n"
    "2. {day_time} {full_name} ({expertise})\n"
    "..."
)

In [50]:
import llama_index.legacy.response_synthesizers

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str2, function_mappings={"context_str": filter_pii_fn}
)


prompts_dict = transform_query_engine.get_prompts()
print(list(prompts_dict.keys()))

transform_query_engine.update_prompts(
    {"query_engine:response_synthesizers": qa_prompt_tmpl}
) # use merging index


['query_transform:hyde_prompt', 'query_engine:response_synthesizer:text_qa_template', 'query_engine:response_synthesizer:refine_template']


### Produce dataframe containing query, model response, and context

In [54]:
def produce_response(query):
    # response = chromadb_query_engine.query(query)
    response = transform_query_engine.query(query)
    return response

def produce_context(query):
    # retrieved_nodes = vector_retriever.retrieve(query)
    retrieved_nodes = automerging_retriever.retrieve(query)
    context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
    return context_str

In [53]:
model_response_df

,query,ref_answer
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...


Produce model response for all queries

In [55]:
model_response_df['model_response'] = model_response_df['query'].apply(produce_response)
time.sleep(5)
model_response_df['context'] = model_response_df['query'].apply(produce_context)

> Merging 1 nodes into parent node.
> Parent node id: 0669f3f3-2e62-44e7-b18b-7c0fcaa2cc49.
> Parent node text: รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหัวใจพิการแต่กำเนิด, หลอดเลือดหัวใจตีบและลิ้นหัวใจ43

> Merging 1 nodes into parent node.
> Parent node id: 923991c1-564f-47be-b184-c097aa8c71eb.
> Parent node text: ศ. พญ.จารุพิมพ์ สูงสว่างพฤหัสบดี 16:00 - 18:00 น. สัปดาห์ที่ 1,2,3,4,5โรคหัวใจในเด็กภาวะหัวใจเต้น...

> Merging 1 nodes into parent node.
> Parent node id: 8359fe5f-5fbf-4eac-a77c-6a1a574872ba.
> Parent node text: ศ.คลินิก นพ.ดำรัส ตรีสุโกศลศุกร์ 08:30 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 13:00 - 15:00 น. สัปดาห์ที...

> Merging 1 nodes into parent node.
> Parent node id: 6b65ec66-0337-48f0-864d-118480114e14.
> Parent node text: ศ.คลินิก นพ.ชาญ ศรีรัตนสถาวรอังคาร 08:30 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:00 - 11:00 น...

> Merging 1 nodes into parent node.
> Parent node id: 34a8ea8f-bc95-48ba-935e-f977fda6538c.
> Parent node text: รศ. นพ.ชุณหเกษม   โชตินัยวัตรกุลอังค

In [56]:
model_response_df

,query,ref_answer,model_response,context
0,ไม่แน่ใจว่าจะเกี่ยวกับโรคหัวใจหรือไม่ครับ อายุ...,ทางเราแนะนำท่านเข้าพบแพทย์ในศูนย์หัวใจ โดยสามา...,จากอาการที่คุณเล่ามา อาจเกิดได้จากหลายสาเหตุ ...,ศ. พญ.จารุพิมพ์ สูงสว่างพฤหัสบดี 16:00 - 18:00...
1,เมื่อสองปีที่แล้วพี่สาวเป็นไทรอยเป็นพิษสุดท้าย...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...,จากอาการที่เล่ามา พี่สาวของคุณอาจมีความเสี่ยง...,"สัปดาห์ที่ 1,2,3,4,5ฝีที่ผิวหนัง, แผลติดเชื้อ,..."
2,ผมตื่นมามีอาการคล้ายสิวหัวช้างที่หลังหูสามเม็ด...,แนะนำให้ผู้ป่วยเข้าพบแพทย์เพื่อประเมินอาการโด...,จากอาการที่เล่ามา แนะนำให้ไปพบแพทย์เฉพาะทางด้...,"สัปดาห์ที่ 1,2,3,4,5ฝีที่ผิวหนัง, แผลติดเชื้อ,..."
3,อายุ 18 ปี ปวดปัสสาวะตอนกลางคืน แต่ไม่ได้ลุกไป...,ท่านสามารถเข้ารับการรักษาได้ที่ศูนย์ทางเดินปัส...,จากอาการที่เล่ามา คล้ายกับอาการของโรคกระเพาะป...,รศ. นพ.ภควัฒน์ ระมาตร์จันทร์ 09:00 - 11:00 น. ...
4,เวียนหัวไม่หมุน โคลงเคลง เพลีย มือเท้าเย็น ไปต...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...,คุณสามารถไปพบแพทย์เฉพาะทางได้ที่ รพ. โดยสามาร...,รศ. นพ.เกรียงไกร ตันติวงศ์โกสีย์nanการผ่าตัดหั...
5,อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามา...,ผู้ป่วยสามารถเข้ารับการวินิจฉัยเบื้องต้นที่ศูน...,จากอาการที่คุณเล่ามา คุณอาจมีภาวะติดเชื้อในระ...,รศ. นพ.เอกรินทร์ โชติกวาณิชย์พุธ 13:00 - 13:30...
6,หายใจมีเสียงหวีด ไอ เหนื่อย มีเสมหะเเหนียว เป็...,ท่านสามารถเข้ารับการวินิจฉัยและรักษาได้ที่ศูนย...,จากอาการที่เล่ามา แนะนำให้พบแพทย์เฉพาะทางด้าน...,พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดิ...
7,มีผื่นคันเป็นขุยๆที่ขา แขนและลำตัว เวลาอากาศเย...,ทางโรงพยาบาลมีศูนย์โรคภูมิแพ้เฉพาะทางที่ท่านสา...,จากอาการที่เล่ามา คล้ายกับโรคสะเก็ดเงิน ซึ่งเ...,รศ. พญ.ปภาพิต ตู้จินดาพฤหัสบดี 09:00 - 12:00 น...
8,ปวดตาทั้งวัน เจอแสงจ้าก็ปวด และทำงานหน้าจอนาน ...,ทางเราแนะนำให้ท่านเข้าพบแพทย์ในศูนย์ตาเฉพาะทาง...,จากอาการที่เล่ามา คุณอาจมีภาวะตาแห้งหรือโรคเก...,รศ. นพ.อดิศักดิ์ ตรีนวรัตน์จันทร์ 18:20 - 19:3...
9,รู้สึกเหมือนมีอะไรเคลื่อนไหวในท้อง ตรงช่วงใต้ล...,ทางเราแนะนำให้รับการวินิจฉัยเบื้องต้นที่ศูนย์อ...,- รศ. นพ.จุลจักร ลิ่มศรีวิไล\n- อ. นพ.วีรวัฒน...,นพ.เอกราช ลิ่วธนมงคลเสาร์ 09:00 - 11:00 น. สัป...


record to csv

In [57]:
fp = "mod02_test_appointments.csv"
model_response_df.to_csv(fp, index=False, mode='w')  # Set index=False to exclude the index column
print("CSV file saved:", fp)

CSV file saved: mod02_test_appointments.csv


### Testing response production

In [58]:
# query1 = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
# query2 = "ดิฉันต้องการนัดพบนักจิตวิทยาเพื่อการปรึกษาเรื่องความเครียด สะดวกมาพบเฉพาะสัปดาห์แรกและสัปดาห์สุดท้ายของเดือน สามารถมาหาหมอได้วันไหนบ้าง"
# query3 = "อยากหาแพทย์เฉพาะทางเพื่อรักษาอาการตกขาวออกมามากปนกับปัสสาวะ มีปัสสาวะแสบและปวดท้องน้อย"
appointment_q = "ผมอยากจะขอนัดหมายกับแพทย์ผู้เชี่ยวชาญด้านโรคเบาหวาน มีวันไหนว่างบ้างในสัปดาห์นี้"

response = produce_response(appointment_q)


> Merging 1 nodes into parent node.
> Parent node id: b623940a-266f-4483-850b-7e1571d5612a.
> Parent node text: ผศ. พญ.สิรินาถ สิรินทร์วราวงศ์พุธ 17:00 - 19:00 น. สัปดาห์ที่ 2,4,5 ศุกร์ 14:00 - 16:00 น. สัปดาห...

> Merging 1 nodes into parent node.
> Parent node id: aa658e96-ca3e-4feb-b36e-299e10e5327d.
> Parent node text: รศ. นพ.จุลจักร ลิ่มศรีวิไลพฤหัสบดี 16:00 - 19:00 น. สัปดาห์ที่ 1,2,3,4,5 ศุกร์ 09:30 - 12:30 น. ส...

> Merging 1 nodes into parent node.
> Parent node id: 3a276b05-8c7b-4d59-a6f2-ea8c2318d95c.
> Parent node text: รศ. นพ.เอกรินทร์ โชติกวาณิชย์พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร์ 09:00 - 12:00 น. สัป...

> Merging 1 nodes into parent node.
> Parent node id: 25616694-8897-450c-bbb7-8ae05a093dfe.
> Parent node text: รศ. นพ.สุนัย ลีวันแสงทองพุธ 14:30 - 15:30 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 12:00 - 14:30 น. สัปดา...

> Merging 1 nodes into parent node.
> Parent node id: f845b397-47b3-4a58-ade2-45a8e6bc5c83.
> Parent node text: อ. นพ.ชวลิต โชติเรืองนภาจันทร์ 1

In [59]:
print("response:" , response)

response:  ผศ. พญ.กุสุมา ไชยสูตร
- วันพุธ เวลา 14:30 - 15:30 น.
- วันพฤหัสบดี เวลา 12:00 - 14:30 น.

ผศ. พญ.สิรินาถ สิรินทร์วราวงศ์
- วันศุกร์ เวลา 14:00 - 16:00 น.


In [60]:
context =  str(produce_context(query))
print("context: " , context)

> Merging 1 nodes into parent node.
> Parent node id: 4ee8b116-f5c1-4d00-8766-4ca303124b09.
> Parent node text: พญ.สิริภัทร ภัทโรดมnanอายุรศาสตร์โรคระบบทางเดินหายใจ535

> Merging 1 nodes into parent node.
> Parent node id: b518b5ca-58c3-4b3b-927f-2e9339730e0a.
> Parent node text: รศ. ดร. พญ.เบญจมาศ ช่วยชูnanอายุรกรรมโรคทรวงอกและทางเดินหายใจ254

> Merging 1 nodes into parent node.
> Parent node id: 41778c05-2960-48fd-990e-1dba25995d4b.
> Parent node text: ศ. นพ.พูนทรัพย์ วงศ์สุรเกียรติ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ โรคปอดและเวชศาสตร์วิกฤต352

> Merging 1 nodes into parent node.
> Parent node id: e12ce163-8514-442d-8e33-6099b584f031.
> Parent node text: นพ.ประทีป รักษาสกุลวงศ์nanอายุรศาสตร์โรคระบบทางเดินหายใจ264

> Merging 1 nodes into parent node.
> Parent node id: 84c16380-a3eb-4910-b78a-45a02ef2a9ae.
> Parent node text: รศ. นพ.บวรฤทธิ์ จักรไพวงศ์อังคาร 07:30 - 08:30 น. สัปดาห์ที่ 1,2,3,4,5การผ่าตัดโรคกระดูกและข้อที่...

> Merging 1 nodes into parent node.
> Parent node id: 3619725

record to csv

In [63]:
import pandas as pd

data = [query, response, context]
df = pd.DataFrame([data], columns=["query", "response", "context"])

fp = "test_results.csv"
df.to_csv(fp, mode='a', header=False, index=False)
print(f"Data saved to {fp}")

Data saved to test_results.csv
